In [1]:
#@title Installing Dependencies (Kaggle GPU case, uncomment if you want to use this one)

import subprocess

subprocess.run(["sudo", "apt-get", "install", "-y", "libgl1-mesa-glx", "libosmesa6"])
subprocess.run(["pip", "install", "-q", "imageio[ffmpeg]"])

print('Installing dm_control...')
!pip install -q dm_control>=1.0.16

%env MUJOCO_GL=osmesa

!echo Installed dm_control $(pip show dm_control | grep -Po "(?<=Version: ).+")
!pip install -q dm-acme[envs]
!mkdir output_videos

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
#@title Importing Libraries
import numpy as np
import collections
import argparse
import os
import yaml
import typing as T
import imageio
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import seaborn as sns
from IPython.display import HTML

import dm_control as dm
import dm_control.suite.swimmer as swimmer
from dm_control.rl import control
from dm_control.utils import rewards
from dm_control import suite
from dm_control.suite.wrappers import pixels

from acme import wrappers

from torch import nn

In [ ]:
#@title Utility code for displaying videos
def write_video(
  filepath: os.PathLike,
  frames: T.Iterable[np.ndarray],
  fps: int = 60,
  macro_block_size: T.Optional[int] = None,
  quality: int = 10,
  verbose: bool = False,
  **kwargs,
):
  """
  Saves a sequence of frames as a video file.

  Parameters:
  - filepath (os.PathLike): Path to save the video file.
  - frames (Iterable[np.ndarray]): An iterable of frames, where each frame is a numpy array.
  - fps (int, optional): Frames per second, defaults to 60.
  - macro_block_size (Optional[int], optional): Macro block size for video encoding, can affect compression efficiency.
  - quality (int, optional): Quality of the output video, higher values indicate better quality.
  - verbose (bool, optional): If True, prints the file path where the video is saved.
  - **kwargs: Additional keyword arguments passed to the imageio.get_writer function.

  Returns:
  None. The video is written to the specified filepath.
  """

  with imageio.get_writer(filepath,
                        fps=fps,
                        macro_block_size=macro_block_size,
                        quality=quality,
                        **kwargs) as video:
    if verbose: print('Saving video to:', filepath)
    for frame in frames:
      video.append_data(frame)


def display_video(
  frames: T.Iterable[np.ndarray],
  filename='output_videos/temp.mp4',
  fps=60,
  **kwargs,
):
  """
  Displays a video within a Jupyter Notebook from an iterable of frames.

  Parameters:
  - frames (Iterable[np.ndarray]): An iterable of frames, where each frame is a numpy array.
  - filename (str, optional): Temporary filename to save the video before display, defaults to 'output_videos/temp.mp4'.
  - fps (int, optional): Frames per second for the video display, defaults to 60.
  - **kwargs: Additional keyword arguments passed to the write_video function.

  Returns:
  HTML object: An HTML video element that can be displayed in a Jupyter Notebook.
  """

  # Write video to a temporary file.
  filepath = os.path.abspath(filename)
  write_video(filepath, frames, fps=fps, verbose=False, **kwargs)

  height, width, _ = frames[0].shape
  dpi = 70
  orig_backend = matplotlib.get_backend()
  matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
  fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
  matplotlib.use(orig_backend)  # Switch back to the original backend.
  ax.set_axis_off()
  ax.set_aspect('equal')
  ax.set_position([0, 0, 1, 1])
  im = ax.imshow(frames[0])
  def update(frame):
    im.set_data(frame)
    return [im]
  interval = 1000/fps
  anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                  interval=interval, blit=True, repeat=False)
  return HTML(anim.to_html5_video())

In [ ]:
_SWIM_SPEED = 0.1
ce_length = 0.8 # total model length

@swimmer.SUITE.add()
def swim(
  n_links=6,
  desired_speed=_SWIM_SPEED,
  time_limit=swimmer._DEFAULT_TIME_LIMIT,
  random=None,
  environment_kwargs={},
):
    """Returns the Swim task for a n-link swimmer."""
    model_string, assets = swimmer.get_model_and_assets(n_links)
    physics = swimmer.Physics.from_xml_string(model_string, assets=assets)

    link_length = ce_length/n_links # length of each body link

    # format head
    physics.named.model.geom_size["visual"] = [0.01, link_length/2, 0]
    physics.named.model.geom_size["inertial"] = [0.01, link_length/2, 0]

    # format first joint
    physics.named.model.jnt_pos["rootx"] = [0, -link_length/2, 0]
    physics.named.model.jnt_pos["rooty"] = [0, -link_length/2, 0]
    physics.named.model.jnt_pos["rootz"] = [0, -link_length/2, 0]

    # format body links
    for i in range(n_links-1):
        physics.named.model.body_pos[f"segment_{i}"] = [0, link_length, 0]
        physics.named.model.geom_size[f"visual_{i}"] = [0.01, link_length/2, 0]
        physics.named.model.geom_size[f"inertial_{i}"] = [0.01, link_length/2, 0]
        physics.named.model.jnt_pos[f"joint_{i}"] = [0, -link_length/2, 0]
    
    task = Swim(desired_speed=desired_speed, random=random)
    return control.Environment(
        physics,
        task,
        time_limit=time_limit,
        control_timestep=swimmer._CONTROL_TIMESTEP,
        **environment_kwargs,
  )


class Swim(swimmer.Swimmer):
    """Task to swim forwards at the desired speed."""
    def __init__(self, desired_speed=_SWIM_SPEED, **kwargs):
        super().__init__(**kwargs)
        self._desired_speed = desired_speed

    def initialize_episode(self, physics):
        super().initialize_episode(physics)
        # Hide target by setting alpha to 0.
        physics.named.model.mat_rgba['target', 'a'] = 0
        physics.named.model.mat_rgba['target_default', 'a'] = 0
        physics.named.model.mat_rgba['target_highlight', 'a'] = 0

    def get_observation(self, physics):
        """Returns an observation of joint angles and body velocities."""
        obs = collections.OrderedDict()
        obs['joints'] = physics.joints()
        obs['body_velocities'] = physics.body_velocities()
        return obs

    def get_reward(self, physics):
        """Returns a smooth reward that is 0 when stopped or moving backwards, and rises linearly to 1
        when moving forwards at the desired speed."""
        forward_velocity = -physics.named.data.sensordata['head_vel'][1]
        return rewards.tolerance(
          forward_velocity,
          bounds=(self._desired_speed, float('inf')),
          margin=self._desired_speed,
          value_at_margin=0.,
          sigmoid='linear',
        )

In [ ]:
""" Renders the current environment state to an image """
def render(env):
    return env.physics.render(camera_id=0, width=640, height=480)

""" Tests a DeepMind control suite environment by executing a series of random actions """
def test_dm_control(env):
    env = wrappers.CanonicalSpecWrapper(env, clip=True)
    env = wrappers.SinglePrecisionWrapper(env)

    spec = env.action_spec()
    timestep = env.reset()
    frames = [render(env)]

    for _ in range(60):
        action = np.random.uniform(low=spec.minimum, high=spec.maximum, size=spec.shape)
        timestep = env.step(action)
        frames.append(render(env))
    return display_video(frames)

In [ ]:
# can sub n_links for any number
# provided two examples
# note that total length is the same but camera zoom is changing (will fix later, head positioning too)
env_6_link = suite.load('swimmer', 'swim', task_kwargs={'random': 1, 'n_links': 6})
env_12_link = suite.load('swimmer', 'swim', task_kwargs={'random': 1, 'n_links': 12})

In [ ]:
test_dm_control(env_12_link)

In [ ]:
test_dm_control(env_6_link)

In [ ]:
# ce_length = 0.8 # total model length
# n_links = 5 # number of body links
# link_length = ce_length/n_links # length of each body link

# # format head
# env.physics.named.model.geom_size["visual"] = [0.01, link_length/2, 0]
# env.physics.named.model.geom_size["inertial"] = [0.01, link_length/2, 0]
# # format first joint
# env.physics.named.model.jnt_pos["rootx"] = [0, -link_length/2, 0]
# env.physics.named.model.jnt_pos["rooty"] = [0, -link_length/2, 0]
# env.physics.named.model.jnt_pos["rootz"] = [0, -link_length/2, 0]
# # format body links
# for i in range(n_links):
#     env.physics.named.model.body_pos[f"segment_{i}"] = [0, link_length, 0]
#     env.physics.named.model.geom_size[f"visual_{i}"] = [0.01, link_length/2, 0]
#     env.physics.named.model.geom_size[f"inertial_{i}"] = [0.01, link_length/2, 0]
#     env.physics.named.model.jnt_pos[f"joint_{i}"] = [0, -link_length/2, 0]

# print(env.physics.named.model.body_pos)
# print(env.physics.named.model.geom_size)
# print(env.physics.named.model.jnt_pos)

In [ ]:
## Example of assembled model_string from swimmer.xml

# model_string = '''
# <mujoco model="swimmer">
#     <include file="./common/visual.xml"/>
#     <include file="./common/skybox.xml"/>
#     <include file="./common/materials.xml"/>

#     <option timestep="0.002" density="3000">
#         <flag contact="disable"/>
#     </option>

#     <default>
#         <default class="swimmer">
#             <joint type="hinge" pos="0 -.075 0" axis="0 0 1" limited="true" solreflimit=".05 1" solimplimit="0 .8 .1" armature="1e-6"/>
#             <default class="inertial">
#                 <geom type="box" size=".001 .075 .01" rgba="1 0 0 0" mass=".01"/>
#             </default>
#             <default class="visual">
#                 <geom type="capsule" size=".01" fromto="0 -.075 0 0 .075 0" material="self" mass="0"/>
#             </default>
#             <site size=".01" rgba="0 0 0 0"/>
#         </default>
#         <default class="free">
#             <joint limited="false" stiffness="0" armature="0"/>
#         </default>
#         <motor gear="5e-4" ctrllimited="true" ctrlrange="-1 1"/>
#     </default>

#     <worldbody>
#         <geom name="ground" type="plane" size="2 2 0.1" material="grid"/>
#         <body name="head" pos="0 0 .05" childclass="swimmer">
#             <light name="light_1" diffuse=".8 .8 .8" pos="0 0 1.5"/>
#             <geom name="head" type="ellipsoid" size=".02 .04 .017" pos="0 -.047 0" material="self" mass="0"/>
#             <geom name="nose" type="sphere" pos="0 -.085 0" size=".004" material="effector" mass="0"/>
#             <geom name="eyes" type="capsule" fromto="-.008 -.079 .005 .008 -.079 .005" size=".004" material="eye" mass="0"/>
#             <camera name="tracking1" pos="0.0 -0.2 0.5" xyaxes="1 0 0 0 1 1" mode="trackcom" fovy="60"/>
#             <camera name="tracking2" pos="-0.9 0.5 0.15" xyaxes="0 -1 0 .3 0 1" mode="trackcom" fovy="60"/>
#             <camera name="eyes" pos="0 -.058 .005" xyaxes="-1 0 0 0 0 1"/>
#             <joint name="rootx" class="free" type="slide" axis="1 0 0" pos="0 -.075 0"/>
#             <joint name="rooty" class="free" type="slide" axis="0 1 0" pos="0 -.075 0"/>
#             <joint name="rootz" class="free" type="hinge" axis="0 0 1" pos="0 -.075 0"/>
#             <geom name="inertial" class="inertial"/>
#             <geom name="visual" class="visual"/>
#             <site name="head"/>
#             <body name="segment_0" pos="0 .15 0">
#                 <geom class="visual" name="visual_0"/>
#                 <geom class="inertial" name="inertial_0"/>
#                 <site name="site_0"/>
#                 <joint name="joint_0" range="-60.0 60.0"/>
#                 <body name="segment_1" pos="0 .15 0">
#                     <geom class="visual" name="visual_1"/>
#                     <geom class="inertial" name="inertial_1"/>
#                     <site name="site_1"/>
#                     <joint name="joint_1" range="-60.0 60.0"/>
#                     <body name="segment_2" pos="0 .15 0">
#                         <geom class="visual" name="visual_2"/>
#                         <geom class="inertial" name="inertial_2"/>
#                         <site name="site_2"/>
#                         <joint name="joint_2" range="-60.0 60.0"/>
#                         <body name="segment_3" pos="0 .15 0">
#                             <geom class="visual" name="visual_3"/>
#                             <geom class="inertial" name="inertial_3"/>
#                             <site name="site_3"/>
#                             <joint name="joint_3" range="-60.0 60.0"/>
#                             <body name="segment_4" pos="0 .15 0">
#                                 <geom class="visual" name="visual_4"/>
#                                 <geom class="inertial" name="inertial_4"/>
#                                 <site name="site_4"/>
#                                 <joint name="joint_4" range="-60.0 60.0"/>
#                             </body>
#                         </body>
#                     </body>
#                 </body>
#             </body>
#         </body>

#         <geom name="target" type="sphere" pos="1 1 .05" size=".1" material="target"/>
#         <light name="target_light" diffuse="1 1 1" pos="1 1 1.5"/>
#     </worldbody>

#     <sensor>
#         <framepos name="nose_pos" objtype="geom" objname="nose"/>
#         <framepos name="target_pos" objtype="geom" objname="target"/>
#         <framexaxis name="head_xaxis" objtype="xbody" objname="head"/>
#         <frameyaxis name="head_yaxis" objtype="xbody" objname="head"/>
#         <velocimeter name="head_vel" site="head"/>
#         <gyro name="head_gyro" site="head"/>
#     </sensor>

#     <actuator>
#         <motor name="motor_0" joint="joint_0"/>
#         <motor name="motor_1" joint="joint_1"/>
#         <motor name="motor_2" joint="joint_2"/>
#         <motor name="motor_3" joint="joint_3"/>
#         <motor name="motor_4" joint="joint_4"/>
#     </actuator>

#     <sensor>
#         <velocimeter name="velocimeter_0" site="site_0"/>
#         <gyro name="gyro_0" site="site_0"/>
#         <velocimeter name="velocimeter_1" site="site_1"/>
#         <gyro name="gyro_1" site="site_1"/>
#         <velocimeter name="velocimeter_2" site="site_2"/>
#         <gyro name="gyro_2" site="site_2"/>
#         <velocimeter name="velocimeter_3" site="site_3"/>
#         <gyro name="gyro_3" site="site_3"/>
#         <velocimeter name="velocimeter_4" site="site_4"/>
#         <gyro name="gyro_4" site="site_4"/>
#     </sensor>
# </mujoco>
# '''